In [1]:
! mkdir ~/.kaggle


In [2]:
!cp ./kaggle.json ~/.kaggle

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c ai-masters-2024-hw-4-license-plate-recognition

100% 809M/809M [00:54<00:00, 18.1MB/s]
100% 809M/809M [00:54<00:00, 15.7MB/s]


In [5]:
!unzip /content/ai-masters-2024-hw-4-license-plate-recognition.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/train/simple/rus_07671_Y583AT35.png  
  inflating: train/train/simple/rus_07672_T627XH35.png  
  inflating: train/train/simple/rus_07673_B538KH50.png  
  inflating: train/train/simple/rus_07674_A999AA777.png  
  inflating: train/train/simple/rus_07675_A383OO197.png  
  inflating: train/train/simple/rus_07676_E422XH32.png  
  inflating: train/train/simple/rus_07677_Y597KK777.png  
  inflating: train/train/simple/rus_07678_B240YY35.png  
  inflating: train/train/simple/rus_07679_Y350PA777.png  
  inflating: train/train/simple/rus_07680_C113YE90.png  
  inflating: train/train/simple/rus_07681_P780AB178.png  
  inflating: train/train/simple/rus_07684_A012TC35.png  
  inflating: train/train/simple/rus_07685_A952PE35.png  
  inflating: train/train/simple/rus_07687_B330PP35.png  
  inflating: train/train/simple/rus_07688_T476PK197.png  
  inflating: train/train/simple/rus_07689_O245BC750.png  
  inflating: 

In [116]:
import os
import shutil

# Пути к исходным папкам
source_folder1 = "train/train/simple"
source_folder2 = "train/train/simple"
destination_folder = "train/data"  # Укажите путь для объединённой папки

# Создание итоговой папки, если она не существует
os.makedirs(destination_folder, exist_ok=True)

# Функция для копирования файлов из одной папки в другую
def merge_folders(source_folder, destination_folder):
    for root, _, files in os.walk(source_folder):
        for file in files:
            source_path = os.path.join(root, file)
            # Итоговый путь к файлу в новой папке
            relative_path = os.path.relpath(source_path, source_folder)
            dest_path = os.path.join(destination_folder, relative_path)

            # Создаём поддиректории, если их ещё нет
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)

            # Копируем файл
            shutil.copy2(source_path, dest_path)

# Объединяем папки
merge_folders(source_folder1, destination_folder)
merge_folders(source_folder2, destination_folder)

print("Папки успешно объединены!")

Папки успешно объединены!


In [94]:
import torchvision.transforms as transforms
import torchvision.datasets   as datasets
import torch.nn.functional    as F
import torchvision.models     as models

import matplotlib.pyplot      as plt
import torch.optim            as optim
import torch.nn               as nn

import torchvision
import colorsys
import random
import torch
import json
import time
import tqdm
import cv2
import os

from torchvision.transforms.functional import convert_image_dtype, resize
from torch.utils.data   import DataLoader
from collections        import namedtuple
from matplotlib.patches import Polygon
from matplotlib         import patches, lines
from PIL                import Image

from torchvision.io import read_image
from string import digits, ascii_uppercase

import albumentations as alb
import numpy          as np
import scipy          as sp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [119]:
PATH_TO_DATA = "./train/data"  # Change to your path with unzipped data

In [182]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

class LicensePlateDataset(Dataset):
   def __init__(self, image_folder, alphabet, transform=None):
       self.image_folder = image_folder
       self.transform = transform
       self.sub_folders = ['complex','simple']

       self.image_files = []
       for sub in self.sub_folders:
         self.image_files += [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

       self.alphabet = alphabet

   def __len__(self):
       return len(self.image_files)

   def __getitem__(self, idx):
       # Получаем имя файла
       filename = self.image_files[idx]
       image_path = os.path.join(self.image_folder, filename)

       # Загружаем изображение
       image = Image.open(image_path).convert('RGB')

       # Извлекаем текст номера из имени файла
       text = filename.rsplit('_', 1)[-1].split('.')[0]  # Убираем расширение

       # Применяем преобразования, если они указаны
       if self.transform:
           image = self.transform(image)

       seq     = self.text_to_seq(text)
       seq_len = len(seq)
       return {"image": image, "text": text, "seq": seq, "seq_len": seq_len, "filename": filename}

   def text_to_seq(self, text):
        """Encode text to sequence of integers.

        Args:
            - String of text.

        Returns:
            List of integers where each number is index of corresponding characted in alphabet + 1.
        """

        # YOUR CODE HERE
        seq = [self.alphabet.find(c) + 1 for c in text]
        # END OF YOUR CODE

        return seq

In [179]:
from torchvision.transforms.v2 import Resize, ToDtype, ToImage, Normalize, Compose
transforms = Compose([
    ToImage(), ToDtype(torch.float32, scale=True), Resize(size=(32,160)),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

abc = "0123456789ABEKMHOPCTYX"  # this is our alphabet for predictions.
full_dataset = LicensePlateDataset(PATH_TO_DATA, abc, transforms)



# Пропорции разделения
val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

# Разделение на тренировочные и валидационные данные
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [184]:
def collate_fn(batch):
    """Function for torch.utils.data.Dataloader for batch collecting.

    Args:
        - batch: List of dataset __getitem__ return values (dicts).

    Returns:
        Dict with same keys but values are either torch.Tensors of batched images or sequences or so.
    """
    torch_images, pil_images, seqs, seq_lens, texts = [], [], [], [], []
    # YOUR CODE HERE
    for item in batch:
        torch_images.append(item["image"]  )
        seqs        .extend(item["seq"        ]  )
        seq_lens    .append(item["seq_len"    ]  )
        texts       .append(item["text"       ]  )
    torch_images = torch.stack(torch_images)
    pil_images   = pil_images
    seqs         = torch.Tensor(seqs    ).int()
    seq_lens     = torch.Tensor(seq_lens).int()
    # END OF YOUR CODE
    return {"torch_images": torch_images, "seq": seqs, "seq_len": seq_lens, "text": texts}



# Создание DataLoader-ов
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)


In [170]:
from torch.nn  import Module, GRU, LSTM, Sequential, Conv2d, Linear

class FeatureExtractor(nn.Module):
    def __init__(self, imgH, leakyRelu=False, PRelu=False):
        super(FeatureExtractor, self).__init__()
        assert imgH % 16 == 0, 'imgH has to be a multiple of 16'

        # YOUR CODE HERE
        ks = [3, 3, 3, 3, 3, 3, 2]
        ps = [1, 1, 1, 1, 1, 1, 0]
        ss = [1, 1, 1, 1, 1, 1, 1]
        nm = [64, 128, 256, 256, 512, 512, 512]

        cnn = nn.Sequential()

        def convRelu(i, batchNormalization=False):
            nIn = 3 if i == 0 else nm[i - 1]
            nOut = nm[i]
            cnn.add_module('conv{0}'.format(i), nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
            if batchNormalization:
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
            if leakyRelu:
                cnn.add_module('relu{0}'.format(i), nn.LeakyReLU(0.2, inplace=True))
            elif PRelu:
                cnn.add_module('relu{0}'.format(i), nn.PReLU())
            else:
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True))

        convRelu(0)
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d(2, 2))  # 64x16x64
        convRelu(1)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d(2, 2))  # 128x8x32
        convRelu(2, True)
        convRelu(3)
        cnn.add_module('pooling{0}'.format(2), nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 256x4x16
        convRelu(4, True)
        convRelu(5)
        cnn.add_module('pooling{0}'.format(3), nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 512x2x16
        convRelu(6, True)  # 512x1x16

        # END OF YOUR CODE
        self.cnn = cnn

    def forward(self, input):
        conv = self.cnn( input )

        b, c, h, w = conv.size()
        conv   = conv.squeeze(2      )
        output = conv.permute(2, 0, 1)  # [w, b, c]

        return output

In [171]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, nL, nIn, nHidden):
        super(BidirectionalLSTM, self).__init__()
        # YOUR CODE HERE
        self.rnn       = nn.LSTM(nIn, nHidden, num_layers=nL, bidirectional=True)
        # END OF YOUR CODE
    def forward(self, input):
        recurrent, _ = self.rnn(input)
        return recurrent

In [172]:
class Encoder(nn.Module):
    def __init__(self, imgH, nh, leakyRelu=False, PRelu=False):
        super(Encoder, self).__init__()
        self.cnn = FeatureExtractor(imgH, leakyRelu=False, PRelu=False)
    def forward(self, input):
        # conv features
        return self.cnn(input)

class Decoder(nn.Module):
    def __init__(self, nl, nh):
        super(Decoder, self).__init__()

        self.rnn_s = BidirectionalLSTM(nl, nh, nh)
    def forward(self, input):
        output = self.rnn_s(input)
        return output

In [173]:
class ConvolutionalRecurrentNeuralNetwork(Module):
    def __init__(self, imgH, nl, nh, nclass, leakyRelu=False, PRelu=False):
        super(ConvolutionalRecurrentNeuralNetwork, self).__init__()
        self.encoder    = Encoder(imgH, nh, leakyRelu=False, PRelu=False)
        self.decoder    = Decoder(nl, nh)
        self.projection = nn.LazyLinear(nclass)
    def forward(self, x):
        y = self.encoder(x)
        y = self.decoder(y)

        T, b, h      = y.size()
        t_rec        = y.view(T * b, h)

        output = self.projection(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output

In [174]:
def pred_to_string(pred, abc):
    seq = []
    for i in range(len(pred)):
        label = np.argmax(pred[i])
        seq.append(label - 1)
    out = []
    for i in range(len(seq)):
        if len(out) == 0:
            if seq[i] != -1:
                out.append(seq[i])
        else:
            if seq[i] != -1 and seq[i] != seq[i - 1]:
                out.append(seq[i])
    out = ''.join([abc[c] for c in out])
    return out

def decode(pred, abc):
    pred = pred.permute(1, 0, 2).cpu().data.numpy()
    outputs = []
    for i in range(len(pred)):
        outputs.append(pred_to_string(pred[i], abc))
    return outputs

In [175]:
!pip install rapidfuzz

In [176]:
nclass     = len(abc) + 1
imgH, imgW = 32, 160
nc, nh     = 3 , 512
nl         = 1

In [223]:
import torch
import numpy as np

import torch
import numpy as np
from rapidfuzz.distance.Levenshtein import distance as lv_dist






def validate(model, val_loader, device):
    model.eval()
    val_loss = 0
    total_levenshtein = 0
    num_samples = 0

    with torch.no_grad():

        progress_bar = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch")
        for j, b in enumerate(progress_bar):
            images = b["torch_images"].to(device)
            seqs_gt = b["seq"]
            seq_lens_gt = b["seq_len"]

            seqs_pred = crnn(images).cpu()
            log_probs = log_softmax(seqs_pred, dim=2)
            seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()

            loss = ctc_loss(log_probs=log_probs,  # (T, N, C)
                            targets=seqs_gt,  # N, S or sum(target_lengths)
                            input_lengths=seq_lens_pred,  # N
                            target_lengths=seq_lens_gt)  # N

            val_loss += loss.item()
            progress_bar.set_postfix(loss = loss.item())

            pred_labels = decode(log_probs, abc)

            # Для каждого примера в батче вычисляем расстояние Левенштейна
            for true_label, pred_label in zip(b['text'], pred_labels):

                # Рассчитываем расстояние Левенштейна
                total_levenshtein += lv_dist(pred_label, true_label)
                num_samples += 1

    # Среднее расстояние Левенштейна
    avg_levenshtein = total_levenshtein / num_samples if num_samples > 0 else 0

    val_loss /= len(val_loader)
    return val_loss, avg_levenshtein

In [226]:
crnn      = ConvolutionalRecurrentNeuralNetwork(imgH, nl, nh, nclass, leakyRelu=False, PRelu=False).to(device)
optimizer = torch.optim.Adam(crnn.parameters(), lr=3e-4, amsgrad=True, weight_decay=1e-4)

In [ ]:
from torch.nn.functional import log_softmax, ctc_loss
import wandb
from tqdm import tqdm

wandb.init(project="HW4")

num_epochs = 10

ACTUALLY_TRAIN = True

crnn.train()
if ACTUALLY_TRAIN:
    for i, epoch in enumerate(range(num_epochs)):
        crnn.train()
        train_loss = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch")
        for j, b in enumerate(progress_bar):
            images = b["torch_images"].to(device)
            seqs_gt = b["seq"]
            seq_lens_gt = b["seq_len"]

            seqs_pred = crnn(images).cpu()
            log_probs = log_softmax(seqs_pred, dim=2)
            seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()

            loss = ctc_loss(log_probs=log_probs,  # (T, N, C)
                            targets=seqs_gt,  # N, S or sum(target_lengths)
                            input_lengths=seq_lens_pred,  # N
                            target_lengths=seq_lens_gt)  # N

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            progress_bar.set_postfix(loss = loss.item())



        # Валидация
        val_loss, avg_levenshtein = validate(crnn, val_loader, device)
        train_loss /= len(train_loader)
        wandb.log({
              "Epoch": epoch + 1,
              "Train Loss": train_loss,
              "Val Loss": val_loss,
              "Val Levenshtein": avg_levenshtein
          })

        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Avg Levenshtein: {avg_levenshtein:.4f}")


else:
    image_train_log = cv2.imread("./resources/train_log.png")
    plt.figure(figsize=(15, 20))
    plt.imshow(image_train_log[:, :, ::-1], interpolation="bilinear")
    plt.axis("off")
    plt.show()

    with open("./data/crnn.pth.tar", "rb") as fp:
        state_dict = torch.load(fp, map_location="cpu")
    crnn.load_state_dict(state_dict)

Epoch,▁▅█
Train Loss,▂▁█
Val Levenshtein,▁▁█
Val Loss,▁▁█
Epoch,3
Train Loss,0.00878
Val Levenshtein,0.04891
Val Loss,0.02509


Epoch 1/10: 100%|██████████| 130/130 [00:09<00:00, 14.07batch/s, loss=0.812]


Epoch 1: Train Loss: 2.0881, Val Loss: 0.5549, Avg Levenshtein: 1.5690


Epoch 2/10:  62%|██████▏   | 80/130 [00:05<00:03, 15.14batch/s, loss=0.03] 